In [4]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [5]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

fatal: destination path 'CV-Parsing-using-Spacy-3' already exists and is not an empty directory.


In [6]:
cv_data=json.load(open("/content/CV-Parsing-using-Spacy-3/data/training/train_data.json"))

In [7]:
len(cv_data)

200

In [8]:
!pip install spacy-transformers
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.8/795.8 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 131.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 120.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [10]:
def get_spacy(file,data):
  for text,annot in tqdm(data):
    doc=nlp.make_doc(text)
    annot=annot["entities"]
    ents=[]
    entity_indices=[]
    for start,end,label in annot:
      skip_entity=False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue
      entity_indices=entity_indices+list(range(start,end))
      try:
        span=doc.char_span(start,end,label=label,alignment_mode="strict")
      except:
        continue

      if span is None:
        err_data=f"{start},{end} {text}" + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass
  return db

In [11]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(cv_data,test_size=0.2)

In [12]:
len(train),len(test)

(160, 40)

In [13]:

db=get_spacy(open('error.txt','w'),train)
db.to_disk("/content/train_data.spacy")

db=get_spacy(open('error.txt','w'),test)
db.to_disk("/content/test_data.spacy")

100%|██████████| 40/40 [00:00<00:00, 71.91it/s]


In [14]:
len(db.tokens)

200

In [17]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 211kB/s]
config.json: 100% 481/481 [00:00<00:00, 3.47MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 5.00MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 4.01MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 11.2MB/s]
2026-01-15 16:46:32.537994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768495592.558155    4963 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768495592.564202    4963 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W

In [16]:
nlp=spacy.load("/content/output/model-best")

OSError: [E050] Can't find model '/content/output/model-best'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
!pip install pyMuPDF

In [ ]:
import sys,fitz
fname='/content/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.docx'
doc=fitz.open(fname)

In [ ]:
text=" "
for page in doc:
  text+=str(page.get_text())


In [ ]:
text=text.strip()

In [ ]:
text=' '.join(text.split())

In [ ]:
text

In [ ]:
doc=nlp(text)
for ent in doc.ents:
    print(ent.text,ent.label_)